In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd drive/My\ Drive/Colab\ Notebooks/SCH_Proposal

/content/drive/My Drive/Colab Notebooks/SCH_Proposal


In [ ]:
!pip3 install -q scispacy
!pip3 install -q https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.3.0/en_core_sci_sm-0.3.0.tar.gz
!pip3 install -q https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.4.0/en_ner_bc5cdr_md-0.4.0.tar.gz

!python3 -m spacy download en

     |████████████████████████████████| 44 kB 2.3 MB/s 
     |████████████████████████████████| 13.5 MB 12.6 MB/s 
     |████████████████████████████████| 71 kB 11.0 MB/s 
     |████████████████████████████████| 6.3 MB 34.0 MB/s 
     |████████████████████████████████| 188 kB 75.5 MB/s 
     |████████████████████████████████| 42 kB 1.4 MB/s 
     |████████████████████████████████| 10.1 MB 58.8 MB/s 
     |████████████████████████████████| 628 kB 62.9 MB/s 
     |████████████████████████████████| 451 kB 54.0 MB/s 
     |████████████████████████████████| 33.1 MB 126 kB/s 
     |████████████████████████████████| 125.1 MB 35 kB/s 
⚠ As of spaCy v3.0, shortcuts like 'en' are deprecated. Please use the
full pipeline package name 'en_core_web_sm' instead.
     |████████████████████████████████| 13.7 MB 7.9 MB/s 
  Attempting uninstall: en-core-web-sm
    Found existing installation: en-core-web-sm 2.2.5
    Uninstalling en-core-web-sm-2.2.5:
      Successfully uninstalled en-core-web-sm-2.2.5

In [ ]:
!pip3 install -q jsonlines

In [ ]:
import pandas as pd
import numpy as np
import pickle as pk
from pprint import pprint
import os

import spacy
import scispacy
from spacy import displacy
import en_core_sci_sm
from scispacy.abbreviation import AbbreviationDetector
from scispacy.linking import EntityLinker
from collections import (OrderedDict,Counter, defaultdict)


from tqdm import tqdm
tqdm.pandas()

import warnings
warnings.filterwarnings('ignore')

import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize
from pprint import pprint
import json
import jsonlines
from ast import literal_eval
from sklearn import metrics
from scipy.spatial import distance

/usr/local/lib/python3.7/dist-packages/spacy/util.py:718: UserWarning: [W094] Model 'en_core_sci_sm' (0.3.0) specifies an under-constrained spaCy version requirement: >=2.3.1. This can lead to compatibility problems with older versions, or as new spaCy versions are released, because the model may say it's compatible when it's not. Consider changing the "spacy_version" in your meta.json to a version range, with a lower and upper pin. For example: >=3.0.7,<3.1.0
  warnings.warn(warn_msg)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


## https://towardsdatascience.com/understanding-faiss-619bb6db2d1a

## Now, onto the actual knowledge retriever (from UMLS etc) component

In [ ]:
!pip3 install -q transformers

     |████████████████████████████████| 3.4 MB 8.2 MB/s 
     |████████████████████████████████| 3.3 MB 51.3 MB/s 
     |████████████████████████████████| 596 kB 66.6 MB/s 
     |████████████████████████████████| 895 kB 62.3 MB/s 
     |████████████████████████████████| 61 kB 637 kB/s 


In [ ]:
import transformers

In [ ]:
!pip3 install -q biobert-embedding==0.1.2

     |████████████████████████████████| 748.9 MB 699 bytes/s 
     |████████████████████████████████| 123 kB 70.0 MB/s 
     |████████████████████████████████| 131 kB 73.3 MB/s 
     |████████████████████████████████| 79 kB 9.3 MB/s 
     |████████████████████████████████| 8.5 MB 70.9 MB/s 
     |████████████████████████████████| 138 kB 72.5 MB/s 
     |████████████████████████████████| 127 kB 68.0 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.11.1+cu111 requires torch==1.10.0, but you have torch 1.2.0 which is incompatible.
torchtext 0.11.0 requires torch==1.10.0, but you have torch 1.2.0 which is incompatible.
torchaudio 0.10.0+cu111 requires torch==1.10.0, but you have torch 1.2.0 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [ ]:
from biobert_embedding.embedding import BiobertEmbedding

In [ ]:
biobert = BiobertEmbedding()

In [ ]:
!pip3 install owlready2

     |████████████████████████████████| 23.8 MB 1.2 MB/s 
  Created wheel for owlready2: filename=Owlready2-0.35-cp37-cp37m-linux_x86_64.whl size=20432818 sha256=63d548646bb68cdfcff75a094ff24fd59d5be6aac1245a9d2a5a2dad9368e6d2
  Stored in directory: /root/.cache/pip/wheels/a6/85/8f/4f254dc1d3b7901c23004544f7214748607d8b0c2c02b3c868
Successfully built owlready2


In [ ]:
from owlready2 import *
from owlready2.pymedtermino2 import *
from owlready2.pymedtermino2.umls import *

In [ ]:
#default_world.set_backend(filename = "pym.sqlite3")
import_umls("../apex-codes/entity_sum/umls-2021AA-full.zip", terminologies = ["ICD10", "SNOMEDCT_US", "CUI"])

Importing UMLS from ../apex-codes/entity_sum/umls-2021AA-full.zip with Python version 3.7.12 and Owlready version 2-0.35...
Full UMLS release - importing UMLS from inner Zip file 2021AA-full/2021aa-1-meta.nlm...
  Parsing 2021AA/META/MRSTY.RRF.gz as MRSTY with encoding UTF-8
  Parsing 2021AA/META/MRRANK.RRF.gz as MRRANK with encoding UTF-8
  Parsing 2021AA/META/MRCONSO.RRF.aa.gz as MRCONSO with encoding UTF-8
  Parsing 2021AA/META/MRCONSO.RRF.ab.gz as MRCONSO with encoding UTF-8
  Parsing 2021AA/META/MRDEF.RRF.gz as MRDEF with encoding UTF-8
Full UMLS release - importing UMLS from inner Zip file 2021AA-full/2021aa-2-meta.nlm...
  Parsing 2021AA/META/MRREL.RRF.aa.gz as MRREL with encoding UTF-8
  Parsing 2021AA/META/MRREL.RRF.ab.gz as MRREL with encoding UTF-8
  Parsing 2021AA/META/MRREL.RRF.ac.gz as MRREL with encoding UTF-8
  Parsing 2021AA/META/MRREL.RRF.ad.gz as MRREL with encoding UTF-8
  Parsing 2021AA/META/MRSAT.RRF.aa.gz as MRSAT with encoding UTF-8
  Parsing 2021AA/META/MRSAT.R

get_ontology("http://PYM/")

In [ ]:
PYM = get_ontology("http://PYM/").load()
default_world.save()
CUI = PYM["CUI"]

### Method to retrieve top-K facts from background KB for each sentence having the named entities extracted. First, tokenize the input_text into sentences and for each named entity, grab the top-K facts and measure semantic similarity using FAISS with the sentence in which the named entity appears in. Ablation study to be conducted with different values of K (5, 10, 15)

In [ ]:
# the dict_data passed here is a json object with article_text, article_abstrat, article_text_named_entities and
# article_abstract_named_entities
# K - number of facts per named entity to be summarized (here the facts are retrieved such that they should be semantically similar to the 
# sentences where the named entity appears in)


def _get_entity_based_facts(dict_data, input = "finding", top_k=5):
  dict_knowledge_facts_final = defaultdict(list)   # to store facts extracted from the background knowledge bases (UMLS, ICD-11, SNOMED-CT)
  lst_knowledge_facts_total = list()

  lst_named_entities_text = list(set(dict_data[f'{input}_entities'].split(' | ')))
  
  input_text = dict_data[input]   # the input_text here is finding or the impression which will be tokenized into sentences
  
  input_text_sentences = sent_tokenize(input_text) # this gives us a list of sentences
  for named_entity in lst_named_entities_text:
    sentences_w_named_entity = [sent for sent in input_text_sentences if named_entity in sent]  # Get sentences with the named entity
  
    try:
      facts_complete = PYM.search(named_entity)

      lst_knowledge_facts = list()
      for fact_str in facts_complete:        
        fact = str(fact_str).split(' # ')[1].strip()   # the actual description linking the named entity pair (disregarding the CUI identifier)
        lst_knowledge_facts.append(fact)
    except:
      continue

    sentence_w_named_entities_embeddings = [biobert.sentence_vector(sent).numpy() for sent in sentences_w_named_entity]
    # avg embedding of all sentences where the named entity appears in
    avg_sentence_w_entities_embedding = np.mean(np.array(sentence_w_named_entities_embeddings), axis=0)   # to be used as a query

    # grab top-k facts with high cosine similarity with the avg_sentence_embedding of all sentencew where the named entity appears in
    dict_fact_cosine = {}
    for fact in lst_knowledge_facts:
      fact_biobert_embedding = biobert.sentence_vector(fact)
      cosine_sim = 1 - distance.cosine(fact_biobert_embedding, avg_sentence_w_entities_embedding)   # cosine distance between fact embedding and avg sent
      dict_fact_cosine[fact] = cosine_sim

    # sort the facts based on cosine_sim and return the top-k facts
    top_k_facts = list({k: v for k, v in sorted(dict_fact_cosine.items(), 
                                                key=lambda item: item[1], reverse=True)}.keys())[:top_k]

    lst_knowledge_facts_total += top_k_facts   # This is for the entire input_text

  knowledge_facts_total_str = " | ".join(lst_knowledge_facts_total)  # Join all the facts with the pipe separator

  return knowledge_facts_total_str


## Main method for the Heart Failure data from UIC

In [ ]:
def main():
  input_fileName = "heart_failure_procedures_w_named_entites.jsonl"
  output_fileName = "heart_failure_procedures_w_named_entites_w_facts.jsonl"

  with open(f"{input_fileName}") as f:
    for idx, line in enumerate(f):
      if idx > 2851:
        try: 
          if idx % 1000 == 0 and idx != 0:
            print("Iteration: ", idx)
          dict_data = json.loads(line)

          finding_facts = _get_entity_based_facts(dict_data, input = "finding")   # call to the knowledge (fact) retriever method--- here it is for source articlce
          impression_facts = _get_entity_based_facts(dict_data, input = "impression")   # call to the knowledge (fact) retriever method--- here it is for source articlce

          dict_1 = {"record_id" : dict_data['record_id'],
                    "procedure_name" : dict_data['procedure_name'],
                    "finding" : dict_data['finding'],
                    "impression" : dict_data['impression'],
                    "finding_entities" : dict_data['finding_entities'],
                    "impression_entities" : dict_data['impression_entities'],
                    "finding_facts" : finding_facts,
                    "impression_facts" : impression_facts
                    }
              
          with jsonlines.open(output_fileName, "a") as writer:
            writer.write(dict_1)
          writer.close()
        except:
          continue

  f.close()

In [ ]:
if __name__ == "__main__":
  main()

## Main Method for the Indiana University benchmark dataset

In [ ]:
def main():
  input_fileName = "NLMCXR_reports_ecgen_radiology_w_named_entites.jsonl"
  output_fileName = "NLMCXR_reports_ecgen_radiology_w_named_entites_w_facts.jsonl"

  with open(f"{input_fileName}") as f:
    for idx, line in enumerate(f):
      try: 
        if idx % 1663 == 0 and idx != 0:
          print("Iteration: ", idx)
        dict_data = json.loads(line)

        finding_facts = _get_entity_based_facts(dict_data, input = "finding")   # call to the knowledge (fact) retriever method--- here it is for source articlce
        impression_facts = _get_entity_based_facts(dict_data, input = "impression")   # call to the knowledge (fact) retriever method--- here it is for source articlce

        dict_1 = {"indication" : dict_data['indication'],
                  "finding" : dict_data['finding'],
                  "impression" : dict_data['impression'],
                  "finding_entities" : dict_data['finding_entities'],
                  "impression_entities" : dict_data['impression_entities'],
                  "finding_facts" : finding_facts,
                  "impression_facts" : impression_facts
                  }
            
        with jsonlines.open(output_fileName, "a") as writer:
          writer.write(dict_1)
        writer.close()
      except:
        continue

  f.close()

In [ ]:
if __name__ == "__main__":
  main()